In [2]:
import pandas as pd
import os

In [10]:
MENU_LATEST =  os.path.dirname(os.getcwd())+'/master_menu/menu.csv'
NUTRITION_SUPERMASTER_LATEST = 'super_master_nutrition/supermaster.csv'
df = pd.read_csv(MENU_LATEST)
df_n = pd.read_csv(NUTRITION_SUPERMASTER_LATEST,converters={'cafeteria_id': lambda x: str(x), 'category': lambda x: str(x)})

df.head(3)

,group,timestamp,name,items,week_num,items_recipe_num,pk
0,afterlunch,2022-03-06 14:30:00Z,"Afterlunch on Sunday, March 06, 2022",[],9,"['102407', '44726.1', '14920', '82258.7']",1000
1,breakfast,2022-03-06 07:30:00Z,"Breakfast on Sunday, March 06, 2022","[1030, 1012, 1022, 1037, 1025]",9,"['2032.23', '18451.22', '616.22', '1125.10', '...",1001
2,dinner,2022-03-06 16:30:00Z,"Dinner on Sunday, March 06, 2022","[1126, 1145, 1147, 1135, 1149, 1169, 1188, 117...",9,"['102098', '4490', '2402', '103772', '33992.8'...",1002


In [11]:
df['items'] = df['items'].apply(lambda x:eval(x))
df['items_recipe_num'] = df['items_recipe_num'].apply(lambda x: eval(x))

In [12]:
DELTA=[]
for index, row in df.iterrows():
    DELTA.append(len(row['items_recipe_num'])-len(row['items']))

In [13]:
row_index = []
for index, row in df.iterrows():
    if len(row['items_recipe_num'])-len(row['items'])>0:
    #if len(row['items']) == 0 and len(row['items_recipe_num'])>0:
        row_index.append(index)

In [14]:
print(row_index)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]


In [15]:
MISSING_DISHES = []
BAD_UNITS_COUNT = []
BAD_UNITS=[]
for i in row_index:
    missing_dishes = df.loc[i, 'items_recipe_num']
    print(missing_dishes)
    u = []
    for h in missing_dishes:
        mm = df_n.loc[df_n[df_n['cafeteria_id'] == h].index, 'portion_volume']
        if mm.empty is True:
            u.append('')
        else:
            try:
                float(mm)
            except:
                u.append(mm.values.tolist()[0])
    if u.count('') == len(u):
        MISSING_DISHES.append(u.count(''))
        BAD_UNITS_COUNT.append(0)
        BAD_UNITS.append(None)
        #MISSING_UNITS.append('Not in FDA Table')
    else:
        MISSING_DISHES.append(u.count(''))
        BAD_UNITS_COUNT.append(len(u)-u.count(''))  
        BAD_UNITS.append(list(filter(('').__ne__, u)))
        #statistic = str(u.count('')) + 'Not in FDA, ' + str(len(u)-u.count('')) + 'non-numeric units'
        #MISSING_UNITS.append(statistic)
    
        

['102407', '44726.1', '14920', '82258.7']
['2032.23', '18451.22', '616.22', '1125.10', '106415', '105032', '68467.1']
['102098', '4490', '2402', '103772', '33992.8', '138982', '117004', '123763.1', '102554', '103757', '10514.6', '9285.5', '118988', '98833', '35685', '67966', '67985', '63438', '14957', '47955.4', '46017.23', '43037.3', '102407', '14920', '44726.1', '82258.7', '103630.1', '103411', '103616.1']
['102346', '102070', '102562', '102370', '103607', '78386.9', '137554.1', '102558', '102648', '999.6', '3978', '10514.6', '63057', '11915']
['36694.2', '8414.39', '45858', '63438', '14957', '47955.4', '46017.23', '43037.3', '102407', '44726.1', '82258.7', '14920', '103643.1', '102800', '103644.1']
['2032.23', '105904', '3619', '5828.2', '108189', '5480.6', '29729.9', '47130.4', '128444']
['103957.3', '11343.5', '128686', '8621', '142444', '103930', '4263.13', '101971', '103274.1', '10518.2', '1462.11', '30876', '135004', '63438', '14957', '47955.4', '46017.23', '43037.3', '102407',

['102487.1', '21742', '82609.2', '105748', '103390', '61177', '135824', '28870', '75761.3', '27288', '117903.1', '5294.16', '35080.10', '111.1', '5541.16', '141888', '138956', '63438', '14957', '47955.4', '46017.23', '43037.3', '102407', '44726.1', '82258.7', '14920', '103643.1', '103619.1', '103586.1', '106089.44', '63969', '1390', '1251', '82259', '62284.1', '62279', '63512.1', '48144.1', '14910', '48144.27', '22792.4', '1748', '43037.1', '43034.1', '46017.25', '63438', '13404.4', '62339', '57425', '13404', '47955.4', '14918', '24644.5', '66851.7', '47957.20', '47957.10', '62369', '62363', '103601']
['106629', '12274.9', '19723.1', '102010', '22741.1', '18183.2', '144407', '103930', '130516', '102964', '105917', '103731.2', '61769.1', '35080.10', '111.1', '63969', '1390', '1251', '82259', '62284.1', '62279', '63512.1', '48144.1', '14910', '48144.27', '22792.4', '1748', '43037.1', '43034.1', '46017.25', '63438', '13404.4', '62339', '57425', '13404', '47955.4', '14918', '24644.5', '668

['22587.7', '103771', '103326', '103333', '102289', '344.11', '123726', '8060.6', '59470', '5912.12', '103834', '33992.71', '102405', '106421']
['102407', '44726.1', '14920', '82258.7']
['2032.23', '105032', '138787', '1125.10', '1178.9', '31169.3', '29729.9', '139524', '19214.2', '139278', '1125.10', '3619', '69553', '36205']
['2402.13', '103399', '123763.2', '140061', '119131.3', '18314.4', '103139', '62161', '102330.1', '102358', '98833', '100472', '129140', '102407', '14920', '44726.1', '82258.7', '103643.1', '102815', '103581.1']
['103695', '16861.2', '110115.2', '102498', '103200', '1105', '105622', '80589.5', '87637', '85896', '2296', '10632.5', '50923.10', '94796.1', '62161', '102330.1']
['49663.1', '21527', '98833', '102407', '44726.1', '82258.7', '14920', '103630.1', '103412', '112307', '4515.3']
['2032.23', '3652', '105032', '138894.6', '8735', '139278', '1011', '139032', '128444', '10895.16', '49508', '101498', '4957', '123728', '5401.9']
['123375', '22087.1', '103772', '10

In [173]:
UNITS_COUNT = [None for i in range(len(df))]
UNITS = [None for i in range(len(df))]
DISHES = [None for i in range(len(df))]
k_unit_c = list(zip(row_index, BAD_UNITS_COUNT))
k_unit = list(zip(row_index, BAD_UNITS))
k_dish = list(zip(row_index, MISSING_DISHES))
for kk in k_unit:
    UNITS[kk[0]] = kk[1]
    
for kk in k_unit_c:
    UNITS_COUNT[kk[0]] = kk[1]
    
for kk in k_dish:
    DISHES[kk[0]] = kk[1]

df['num_missing_Dish'] = DELTA
df['num_bad_units'] = UNITS_COUNT
df['num_missing_FDA'] = DISHES
df['bad_units'] = UNITS

In [174]:
df.to_csv('Menu_Study_Analysis.csv')